# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive
drive.mount('/content/drive/')
#%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Residency_11/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

In [0]:
train_bodies = pd.read_csv(project_path+'train_bodies.csv')

In [0]:
train_stances = pd.read_csv(project_path+'train_stances.csv')

In [0]:
train_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [0]:
train_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
dataset = train_bodies.merge(train_stances,on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [0]:
dataset.head(20)

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated
5,0,A small meteorite crashed into a wooded area i...,Enormous 20-stone catfish caught with fishing ...,unrelated
6,0,A small meteorite crashed into a wooded area i...,Italian catches huge wels catfish; is it a rec...,unrelated
7,0,A small meteorite crashed into a wooded area i...,Not coming to a store near you: The pumpkin sp...,unrelated
8,0,A small meteorite crashed into a wooded area i...,One gunman killed in shooting on Parliament Hi...,unrelated
9,0,A small meteorite crashed into a wooded area i...,Soldier shot at war memorial in Canada,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tkzer = Tokenizer(num_words=MAX_NB_WORDS, lower=False)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
tkzer.fit_on_texts(dataset.articleBody)
tkzer.fit_on_texts(dataset.Headline)

In [0]:
index = tkzer.word_index

In [0]:
index['A']

43

In [0]:
#tkzer.document_count

In [0]:
#tkzer.word_docs

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [0]:
len(tkzer.word_index)

33148

In [0]:
#sent_list.keys()

### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = dataset.articleBody

In [0]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [0]:
from nltk.tokenize import sent_tokenize

In [0]:
articles = []
count = 0
for text in texts:
  sents = sent_tokenize(text)
  articles.append(sents)

In [0]:
len(articles)

49972

## Check 2:

first element of texts and articles should be as given below. 

In [0]:
texts[500]

'DUBAI - A prominent Saudi Arabian cleric has whipped up controversy by issuing a religious ruling forbidding the building of snowmen, describing them as anti-Islamic.\n\nAsked on a religious website if it was permissible for fathers to build snowmen for their children after a snowstorm in the country\'s north, Sheikh Mohammed Saleh al-Munajjid replied: "It is not permitted to make a statue out of snow, even by way of play and fun."\n\nQuoting from Muslim scholars, Sheikh Munajjid argued that to build a snowman was to create an image of a human being, an action considered sinful under the kingdom\'s strict interpretation of Sunni Islam.\n\n"God has given people space to make whatever they want which does not have a soul, including trees, ships, fruits, buildings and so on," he wrote in his ruling.\n\nThat provoked swift responses from Twitter users writing in Arabic and identifying themselves with Arab names.\n\n"They are afraid for their faith of everything ... sick minds," one Twitte

In [0]:
articles[500]

['DUBAI - A prominent Saudi Arabian cleric has whipped up controversy by issuing a religious ruling forbidding the building of snowmen, describing them as anti-Islamic.',
 'Asked on a religious website if it was permissible for fathers to build snowmen for their children after a snowstorm in the country\'s north, Sheikh Mohammed Saleh al-Munajjid replied: "It is not permitted to make a statue out of snow, even by way of play and fun."',
 "Quoting from Muslim scholars, Sheikh Munajjid argued that to build a snowman was to create an image of a human being, an action considered sinful under the kingdom's strict interpretation of Sunni Islam.",
 '"God has given people space to make whatever they want which does not have a soul, including trees, ships, fruits, buildings and so on," he wrote in his ruling.',
 'That provoked swift responses from Twitter users writing in Arabic and identifying themselves with Arab names.',
 '"They are afraid for their faith of everything ... sick minds," one T

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [0]:
#data = np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH), dtype=int)

In [0]:
#for article in articles:
#  words_sents = []
#  for sent in article[0:MAX_SENTS]:
#    #print(sent)
#    words_sents.append(text_to_word_sequence(sent,lower=False))
#  words_articles.append(words_sents)
  

In [0]:
#from sklearn.feature_extraction.text import CountVectorizer

In [0]:
#cvect = CountVectorizer()

In [0]:
#cvect

In [0]:
data = np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH), dtype=int)
count_articles = 0
for article in articles:
  count_sent = 0
  for sent in article[0:MAX_SENTS]:
    sent = article[count_sent]
    words = text_to_word_sequence(sent,lower=False) 
    count_word = 0
    for word in words[0:MAX_SENT_LENGTH]:
      #print(word)
      data[count_articles,count_sent, count_word] = index[word] 
      count_word +=1
    count_sent +=1 
  count_articles +=1

### Check 3:

Accessing first element in data should give something like given below.

In [0]:
data[0,:,:]

array([[   43,   513,   509,  7875,    75,     3,  3923,   335,     5,
         4087,   403,     4,  1497,  3089,     1,    90,    13,   464,
            0,     0],
       [ 5543,    92,  1059,     3,  3088,  3047,     7,   180,     3,
         1203,  1133,  2644,   763,   158,     1,  3378,   588,     1,
          741,   533],
       [ 1514,  1064,  4354,  2420,    13,     3,  3071,  3437,    19,
            1,    90,     2,  2027,     1,   535,  2150,    18,     8,
            3,  3231],
       [  250,  4059,  1014,   193,  2652,    41,  7381,  1734,  1297,
            5, 15156, 20816,     1,   881,    32,   734,  4193,   145,
           87,     0],
       [ 2420,    13,  1621,    38,  1103,   655,   917,     2,   361,
          297,  1986,     5,  4739,    71,   494,     0,     0,     0,
            0,     0],
       [   11,   763,   180,    19,     1,   509,    30,     3,  8104,
            4,  2220,  1297,     6,     3,  5659,     4,  1203,  1297,
           13,  3488],
       [  

In [0]:
data.shape

(49972, 20, 20)

In [0]:
#Intended Output
data[0, :, :]

array([[    3,   487,   474,  7113,    79,     3,  3687,   325,     5,
         4200,   361,     4,  1525,  2913,     1,    89,    12,   451,
            0,     0],
       [  743,    96,  1044,     3,  2814,  1759,     7,   186,     3,
         1219,  1070,  1987,   736,   154,     1,  2990,   458,     1,
          543,   232],
       [   89,  1052,  4057,  2314,    12,     3,  1073,  3248,    19,
            1,    89,     2,  1751,     1,   518,  1980,    15,     9,
            3,  2879],
       [  182,  3691,   976,   196,  2515,    42,  6688,  1691,  1227,
            5, 13011, 17379,     1,   762,    30,   722,  3931,    66,
           87,     0],
       [ 2314,    12,  1882,    38,  1076,   346,   793,     2,   356,
          261,  1782,     5,  4396,    67,   486,     0,     0,     0,
            0,     0],
       [    1,   736,   186,    19,     1,   474,    32,     3,  7307,
            4,  2122,  1227,     6,     3,  5195,     4,  1219,  1227,
           12,  3308],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = dataset.Headline
articles_heading = []
count = 0
for text in texts_heading:
  sents = sent_tokenize(text)
  articles_heading.append(sents)
 
data_heading = np.zeros((len(articles_heading), MAX_SENTS, MAX_SENT_LENGTH), dtype=int)
count_articles = 0
for article in articles_heading:
  count_sent = 0
  for sent in article[0:MAX_SENTS]:
    sent = article[count_sent]
    words = text_to_word_sequence(sent,lower=False) 
    count_word = 0
    for word in words[0:MAX_SENT_LENGTH]:
      #print(word)
      data_heading[count_articles,count_sent, count_word] = index[word] 
      count_word +=1
    count_sent +=1 
  count_articles +=1

In [0]:
data_heading[0,:,:]

array([[ 4142,   215,   498,  7790,   190,    37,  1425, 13017,    24,
          402,  2104,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  

In [0]:
data_heading.shape

(49972, 20, 20)

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
import pandas as pd

In [0]:
labels = pd.get_dummies(dataset.Stance) 

In [0]:
labels.shape

(49972, 4)

In [0]:
labels.head(20)

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
5,0,0,0,1
6,0,0,0,1
7,0,0,0,1
8,0,0,0,1
9,0,0,0,1


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [0]:
print('Shape of data tensor:', data.shape)
print('Shape of data_heading tensor:', data_heading.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of data_heading tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


In [0]:
data_final = np.hstack((data,data_heading))

In [0]:
data_final.shape

(49972, 40, 20)

### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels.iloc[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
x_train = data[0 : int(len(data) * 0.8) + 1, :, :]
x_val = data[int(len(data) * 0.8) + 1 : , :, :]

x_heading_train = data_heading[0 : int(len(data_heading) * 0.8) + 1, :, :]
x_heading_val = data_heading[int(len(data_heading) * 0.8) + 1 : , :, :]

y_train = labels.iloc[0:int(len(data) * 0.8) + 1, :]
y_val = labels.iloc[int(len(data) * 0.8) + 1 : , :]

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [0]:
print(x_train.shape)
print(x_heading_train.shape)
print(y_train.shape)

print(x_val.shape)
print(x_heading_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 20, 20)
(9994, 4)


Below changes for Sequential model

In [0]:
X_train_article_reshaped = x_train.reshape(39978, 400)
X_train_head_reshaped = x_heading_train.reshape(39978, 400)
X_test_article_reshaped = x_val.reshape(9994, 400)
X_test_head_reshaped = x_heading_val.reshape(9994, 400)

In [0]:
X_train_article_reshaped = X_train_article_reshaped.tolist()

In [0]:
X_train_head_reshaped = X_train_head_reshaped.tolist()

In [0]:
X_test_article_reshaped = X_test_article_reshaped.tolist()
X_test_head_reshaped = X_test_head_reshaped.tolist()

In [0]:
df_final_train = pd.DataFrame()
df_final_train['X_article'] = X_train_article_reshaped
df_final_train['X_heading'] = X_train_head_reshaped 

In [0]:
df_final_train.shape

(39978, 2)

In [0]:
df_final_test = pd.DataFrame()
df_final_test['X_article'] = X_test_article_reshaped
df_final_test['X_heading'] = X_test_head_reshaped 

In [0]:
df_final_test.shape

(9994, 2)

### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [0]:
vocab_size=len(tkzer.word_index)

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in tkzer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense,Concatenate,Flatten,Dropout
import keras
from keras.models import Model, Sequential
from keras import layers


### Sequential Model

In [0]:
model1 = Sequential()
model1.add(Dense(128, activation='relu', input_shape = (1,400)))
model1.add(Dropout(0.2))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))
model1.compile(loss=keras.losses.categorical_crossentropy,
              #optimizer=keras.optimizers.Adadelta(),
              optimizer='adam',
              metrics=['accuracy'])


In [0]:
df_final_train.

In [0]:
model1.fit(df_final_train,y_train, batch_size=32,epochs = 10)

ValueError: ignored

### Model (LSTM and Embedding)

In [0]:
article_input = Input(shape=(None,), dtype='int32', name='article_input')
head_input = Input(shape=(None,), dtype='int32', name='head_input')


In [0]:
shared_embed = Embedding(len(tkzer.word_index),100,weights=[embedding_matrix],trainable=False)


In [0]:
article_embed = shared_embed(article_input)
head_embed = shared_embed(head_input)


In [0]:
article_lstm = LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='article_lstm')(article_embed)
head_lstm = LSTM(64,dropout=0.2, recurrent_dropout=0.2, name='head_lstm')(head_embed)


In [0]:
conc = layers.concatenate([article_lstm,head_lstm],axis=-1)


In [0]:
dense = Dense(100,activation='relu')(conc)
dense = Dropout(0.3)(dense)
dense = Dense(4,activation='softmax')(dense)


In [0]:
model = Model(inputs=[article_input,head_input], outputs=[dense])


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
article_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
head_input (InputLayer)         (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    3314800     article_input[0][0]              
                                                                 head_input[0][0]                 
__________________________________________________________________________________________________
article_lstm (LSTM)             (None, 64)           42240       embedding_2[0][0]          

### Compile and fit the model

In [0]:
model.fit([x_train.reshape(39978, 400),x_heading_train.reshape(39978, 400)],[y_train],validation_data = ([x_val.reshape(9994,400),x_heading_val.reshape(9994,400)],y_val),epochs=4, batch_size=128,verbose = True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 39978 samples, validate on 9994 samples
Epoch 1/4
39978/39978 [==============================] - 412s 10ms/step - loss: 0.8344 - acc: 0.7270 - val_loss: 0.7835 - val_acc: 0.7354
Epoch 2/4
39978/39978 [==============================] - 398s 10ms/step - loss: 0.7911 - acc: 0.7310 - val_loss: 0.7693 - val_acc: 0.7389
Epoch 3/4
39978/39978 [==============================] - 393s 10ms/step - loss: 0.7771 - acc: 0.7359 - val_loss: 0.7656 - val_acc: 0.7426
Epoch 4/4
39978/39978 [==============================] - 388s 10ms/step - loss: 0.7639 - acc: 0.7396 - val_loss: 0.7486 - val_acc: 0.7449


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)